## Inspect Provided Data

This notebook inspects and plots the data provided in `data/`.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from __future__ import annotations

from collections.abc import Sequence

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io as spio
import scipy.stats

from network_utils import (
    create_56bus,
    create_RX_from_net,
    calc_voltage_profile)

from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')

# hide top and right splines on plots
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

In [3]:
time_ticks =  [0      ,    2400,    4800,    7200,    9600,   12000,   14400]
time_labels = ['0h', '4h', '8h', '12h', '16h', '20h', '24h']
# time_labels = ['00:00', '04:00', '08:00', '12:00', '16:00', '20:00', '24:00']

vmin = 11.4  # kV
vmax = 12.6

In [4]:
def savefig(fig: plt.Figure, file_path: str) -> None:
    fig.savefig(file_path, dpi=200, pad_inches=0, bbox_inches='tight')

## Load aggregate data

In [ ]:
# load the mat files
solar = spio.loadmat('data/PV.mat', squeeze_me=True)
aggr_p = spio.loadmat('data/aggr_p.mat', squeeze_me=True)
aggr_q = spio.loadmat('data/aggr_q.mat', squeeze_me=True)
pq_fluc = spio.loadmat('data/pq_fluc.mat', squeeze_me=True)

# for mat in [actual_solar, actual_p, actual_q, pq_fluc]:
#     display(mat)

# all of the `.mat` files have only 1 key each
solar = solar['actual_PV_profile']  # shape [14421]
aggr_p = aggr_p['p']  # shape [14421]
aggr_q = aggr_q['q']  # shape [14421]
pq_fluc = pq_fluc['pq_fluc']  # shape [55, 2, 14421]

In [ ]:
# for reference: there are 14,400 seconds in a 24-hour period
for x in [solar, aggr_p, aggr_q, pq_fluc]:
    print(x.dtype, x.shape)

## Explore aggregated data

Aggregated data is the sum of the per-bus data. Note that the aggregated active power load does not include the effect of solar generation. That is, the net active power load is:

```python
net_active_load = aggr_p - solar
```

In [ ]:
df = pd.DataFrame({'solar': solar, 'p': aggr_p, 'q': aggr_q})
df.describe()

In [ ]:
# Recreate Fig7 from Qu and Li (2020), equivalent to Fig5 (left) from  Shi et al. (2021)
fig, ax = plt.subplots(figsize=(5, 4))
ax.plot(solar, label='Solar (MW)')
ax.plot(aggr_p, label='Active Load (MW)')
ax.plot(aggr_q, label='Reactive Load (MVar)')

ax.legend(loc='upper left')
ax.set(xlabel='Time (Hour)', ylabel='Power')
ax.set(xticks=time_ticks, xticklabels=time_labels)
plt.show()

In [ ]:
# plot only solar and active load
fig, ax = plt.subplots(figsize=(4,3), tight_layout=True)
ax.plot(solar, label='Solar (MW)')
ax.plot(aggr_p, label='Active Load (MW)', color='tab:orange')
ax.plot(aggr_p - solar, ' ', label='Net load (MW)', color='tab:green')

ax.set(ylabel='Power (MW)')
ax.set(xticks=time_ticks, xticklabels=time_labels)
savefig(fig, 'plots/sce56_solar_and_load.svg')

ax.plot(aggr_p - solar, ':', label='Net load (MW)', color='tab:green')
savefig(fig, 'plots/sce56_net_load.svg')

## Explore per-bus data

Whereas the aggregate `p` and `q` were specified as "load", the per-bus data is specified as "power injection". That is, the signs are opposite.

Furthermore, per-bus `p` is _net_ active power injection (solar generation - load). Hence,

```python
solar - aggr_p = all_p.sum(axis=0)
```

In [ ]:
all_p = pq_fluc[:, 0]  # shape [n, T]
all_q = pq_fluc[:, 1]  # shape [n, T]

assert np.allclose(solar - all_p.sum(axis=0), aggr_p)
assert np.allclose(-all_q.sum(axis=0), aggr_q)

In [ ]:
p_df = pd.DataFrame(all_p.T)  # rows = time, cols = buses
q_df = pd.DataFrame(all_q.T)

min_p = p_df.min()
max_p = p_df.max()

fig, axs = plt.subplots(1, 2, figsize=(8, 4), sharey=True, tight_layout=True)
ax = axs[0]
ax.set_title('active power injection')
min_p.plot(kind='bar', ax=ax, color='blue', label='min')
max_p.plot(kind='bar', ax=ax, color='orange', label='max')

for i in max_p.index:
    val = max_p.loc[i]
    if val > 0.1:
        ax.annotate(f'{val:.2f}\n({i})', (i - 2, val + 0.1))

ax = axs[1]
ax.set_title('reactive power injection')
q_df.min().plot(kind='bar', ax=ax, color='blue', label='min')
q_df.max().plot(kind='bar', ax=ax, color='orange', label='max')

for ax in axs:
    ax.set(xlabel='bus ID', xticks=range(0, 55, 5))
    ax.legend()

plt.show()

Plot buses whose active power injections are most and least correlated with solar

In [ ]:
pv_corr = pd.Series(
    data=[scipy.stats.pearsonr(p, solar)[0] for p in all_p],
    name='corr')
pv_corr.sort_values(ascending=False, inplace=True)
pv_corr.index += 1
display(pv_corr.head())
display(pv_corr.tail())

fig, ax = plt.subplots(1, 1, tight_layout=True)
ax.plot(solar)
for i in pv_corr.index[:5]:
    ax.plot(all_p[i-1], label=f'{i}, r={pv_corr.loc[i]:.2f}')
for i in pv_corr.index[-5:]:
    ax.plot(all_p[i-1], label=f'{i}, r={pv_corr.loc[i]:.2f}')
ax.legend(title='node ID')
ax.set(xlabel='Time', ylabel='Power (MW)', title='active power injection')
ax.set_xticks(time_ticks)
ax.set_xticklabels(time_labels)
plt.show()

In [ ]:
pv_nodes_gq = np.array([2, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 19, 20, 23, 25, 26, 32])
pv_corr.loc[pv_nodes_gq - 1]

In [ ]:
net = create_56bus()
R, X = create_RX_from_net(net)
v_sq = calc_voltage_profile(X, R, p=all_p, qe=all_q, qc=0, v_sub=12**2)
v = np.sqrt(v_sq)

In [ ]:
def plot_voltage_and_injections(indices: Sequence[int], offset: int = 0) -> None:
    fig, axs = plt.subplots(1, 3, figsize=(12, 4), tight_layout=True)

    for i in np.array(indices) - offset:
        axs[0].plot(v[i], label=f'bus {i+offset}')
        axs[1].plot(all_p[i])
        axs[2].plot(all_q[i])

    for ax in axs:
        ax.set(xticks=time_ticks, xticklabels=time_labels)

    ax = axs[0]
    ax.axhline(vmin, ls='--')
    ax.axhline(vmax, ls='--')
    ax.legend()
    ax.set_title('Voltage Profile without Controller')

    axs[1].set_title('Active power injection ($p$, MW)')
    axs[2].set_title('Reactive power injection ($q^e$, MVar)')

    plt.show()

In [ ]:
# Recreate Fig8 (right) in Qu and Li (2020)
# - they count the substation as bus 1
# - we count the substation as bus -1
index = [9, 19, 22, 31, 40, 46, 55]
plot_voltage_and_injections(indices=index, offset=2)

In [ ]:
# Recreate Fig5 (middle) in Shi et al. (2021)
# - like us, they count the substation as bus -1
index = [8, 10, 18, 21, 29, 45, 54]
plot_voltage_and_injections(indices=index)

Plot nodes with largest violations above and below the limits

In [ ]:
pos_maxviolate_node = v.max(axis=1).argmax()
neg_maxviolate_node = v.min(axis=1).argmin()
print(f'Node: {pos_maxviolate_node}, Voltage: {v[pos_maxviolate_node].max()}')
print(f'Node: {neg_maxviolate_node}, Voltage: {v[neg_maxviolate_node].min()}')

In [ ]:
index = [17, 35]
plot_voltage_and_injections(indices=index)

In [16]:
solar_orig = spio.loadmat('orig_data/pvavail20150908_2.mat', squeeze_me=True)
actual_p = spio.loadmat('data/aggr_p.mat', squeeze_me=True)
actual_q = spio.loadmat('data/aggr_q.mat', squeeze_me=True)

/Users/jing/opt/anaconda3/lib/python3.7/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [21]:
solar_orig['PVavail']

array([('P5076423', 85, 75, 121.6, array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])),
       ('P5073132', 121, 150, 100, array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])),
       ('P5620916', 71, 75, 0, 0, 0),
       ('P5446952', 48, 750, 481, array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])),
       ('P5078992', 125, 150, 50.2, array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])),
       ('4471502E', 41, 25, 0, 0, 0), ('P5506503', 28, 750, 0, 0, 0),
       ('4801319E', 25, 300, 0, 0, 0),
       ('P5398618', 86, 300, 188.1, array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])),
       ('1680892E', 3, 15, 0, 0, 0),
       ('P5512210', 44, 75, 106.1, array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])),
       ('P5320414', 8, 300, 61.6, array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])),
       ('P5072219', 115, 1000, 7

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Thu May 27 21:57:41 2021',
 '__version__': '1.0',
 '__globals__': [],
 'q': array([1.36554249, 1.3643562 , 1.3631536 , ..., 1.89689961, 1.89485952,
        1.89542297])}